In [1]:
import pandas as pd
import numpy as np
import string
import matplotlib.pyplot as plt
from matplotlib.pyplot import plot
from collections import Counter
from Chemometrics.basic import psave,pload,train_and_evaluate
%matplotlib inline

plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

In [2]:
X = pload('./中汽15个部件各组分浓度.p').fillna(0)
y = pload('./零部件气味等级.p')
to = pload('./整车数据.p')

FileNotFoundError: [Errno 2] No such file or directory: './中汽15个部件各组分浓度.p'

In [ ]:
import codecs

In [ ]:
_ = X.iloc[[1,2],[2,3]]
_.index.name='部件名称'
X_json = _.to_json(orient='table',force_ascii=False)
X_json

In [ ]:
with open('y.json','w',encoding='utf-8') as file:
    y.to_json(file,orient='table',force_ascii=False)

In [ ]:
import json
json.dumps(json.loads(y.to_json(orient='table',force_ascii=False)),ensure_ascii=False)

In [ ]:
json.dumps?

# 气味相图

In [ ]:
np.random.seed(0)
ts = np.sort(abs(np.random.randn(10))).reshape(2,-1).T
i = iter(string.ascii_lowercase[:len(ts)])
ts1 = pd.DataFrame(ts,index=i,columns=['log(odt)','log(npt)'])
ts1

In [ ]:
with open('ts1.json','w') as file:
    ts1.to_json(file,orient='table',force_ascii=False)

In [ ]:
json.loads(ts1.to_json(orient='table',force_ascii=False))['schema']

In [ ]:
k = 8/np.diff(ts1[['log(odt)','log(npt)']],axis=1)#斜率
b = 1-ts1['log(odt)']*k.flatten() #截距
ts1['k'] = k
ts1['b'] =b
ts1

In [ ]:
# 气味相图
lines = plot(np.array([1,9]*5).reshape(-1,2).T,ts1[['log(odt)','log(npt)']].values.T,'o-')
plt.legend(lines, ts1.index.values.tolist())

In [ ]:
import numpy as np
np.array([1,9]*5).reshape(-1,2).T

# 部件名称不对应

~~~py
# 对应
yi = ['仪表板',
 '前围隔音垫',
 '前排座椅',
 '后围内饰板',
 '后排座椅',
 '前地板隔音垫',
 '地毯',
 '密封条',
 '空调主机',
 '行李厢地毯',
 '行李舱左右装饰板',
 '行李厢隔板装饰板',
 '车门内饰板',
 '遮阳板',
 '顶棚']
y.sort_index(inplace=True)
y.index = yi
psave(y,'./零部件气味等级.p')
~~~

In [ ]:
y = pload('./零部件气味等级.p')
X = X.sort_index()
y = y.sort_index()

# 等级预测

In [ ]:
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.cross_decomposition import PLSRegression

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.33)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
parts = []
parts += [car['partsTree']['CurrentPart']]
parts

In [ ]:
'CurrentPart' in car.keys()

In [ ]:
car_json = json.dumps(car)
part = json.loads(car_json)
def tree2parts(part,parts=[]):
    """
    遍历递归整车树，找到所有的部件
    """
    if 'CurrentPart' in part.keys():
        parts += [part['CurrentPart']]
        if 'Children' in part.keys():
            part = part['Children']
            parts = tree2parts(part,parts)
    return parts        
pd.DataFrame(tree2parts(part['partsTree']))

In [ ]:
pd.DataFrame(tree2parts(part['partsTree']))

In [ ]:
params = {'n_components':range(2,5)}

In [ ]:
regor = GridSearchCV(PLSRegression(),params,cv=10)
regor.fit(X_train,y_train)

In [ ]:
a = pd.DataFrame([y_test.values,regor.predict(X_test).reshape(-1)]).T.set_index(y_test.index)
a.columns = ['真实值','模型预测值']
a

In [ ]:
import seaborn as sns
sns.regplot(y_test,regor.predict(X_test).reshape(-1))
plt.xlabel('measured')
plt.ylabel('predicted')

# 气味溯源

In [ ]:
plt.bar(range(len(to)),to.sort_values(ascending=False))

In [ ]:
# 零部件
a = X.T.iloc[:,0]
a = a[to.index]
plt.bar(range(len(a)),a)

In [ ]:
def find_source(total,parts,by,is_plot=True):
    """
    找出total的by在各parts中的来源
    """
    by = total.index[total.index.isin(by)]
    influence = total/total.sum()
    not_in = by[~by.isin(parts.columns)].tolist()
    print('所有部件中不存在'+str(not_in))
    res = parts.loc[:,by].fillna(0).dot(influence[by]).sort_values(ascending=False)
    if is_plot:
        sns.barplot(res,res.index,orient='h')
    return res

In [ ]:
a = find_source(to,X,sr)

# 整车有的组分，所有零部件中都没有